In [1]:
import urllib
import pandas as pd
import numpy as np
import datetime
import string
import pymysql as mdb
maketables=False
maketables=True

In [2]:
topicsfile="/Users/abramvandergeest/Dropbox/insight_work/topic_list.txt"
f=open(topicsfile)
topics=f.readlines()
f.close()

In [3]:
con = mdb.connect('localhost', 'abram.ghost', '', 'wikidata')
####I NEED TO SET THIS TO A NON-GITHUB AUTHENTICATION LOCATION AND CHANGE PASSWORD

#I should also create the  database if it doesn't exist

In [4]:
if maketables==True:
    #Makes topics table
    with con:
        cur = con.cursor()
        cur.execute("DROP TABLE IF EXISTS `topics`;")
        str="CREATE TABLE `topics` (`Id` INT PRIMARY KEY AUTO_INCREMENT, `topic_label` VARCHAR(64), `topic_string` VARCHAR(64));"
        cur.execute(str)
    
    #Populate topics table
    with con:
        for topic in topics:
            label="".join(topic.lower().split())
            string=topic.rstrip()
            sql="INSERT INTO `topics` (`topic_label`,`topic_string`) VALUES (%s,%s);"
            cur.execute(sql,(label,string))
        
    #Makes page_views table
    with con:
        cur = con.cursor()
        cur.execute("DROP TABLE IF EXISTS `page_views`;")
        str="CREATE TABLE `page_views` (`Id` INT PRIMARY KEY AUTO_INCREMENT, `topic_id` INT, `count` INT, `date` VARCHAR(64));"
        cur.execute(str)    

In [14]:
def get_unzip(d,fo,fo2):
    wikiurl="http://dumps.wikimedia.org/other/pagecounts-raw/%4d/%4d-%02d/"%(d.year,d.year,d.month)
    gzfile="pagecounts-%4d%02d%02d-%02d0000.gz"%(d.year,d.month,d.day,d.hour)
    print gzfile
    fn=wikiurl+gzfile
    urllib.urlretrieve(fn,fo)
    !gunzip -c $fo>$fo2
    return

def csv_to_reduceddf(fo2):
    allwiki=pd.read_csv(fo2,delim_whitespace=True)
    allwiki.columns=['lang','article','views','size']
    enwiki=allwiki[allwiki['lang']=='en']
    enwiki=enwiki[enwiki['size']>=50000]
    return(enwiki)

In [17]:
#creating a dataframe with all english pages of a large size
print "starting"
year=2015
month=3
day=1
hour=16
fo="/Users/abramvandergeest/wikidata/temp.gz"
fo2="/Users/abramvandergeest/wikidata/temp.txt"
d=datetime.datetime(year,month,day,hour)
dt=datetime.timedelta(days=+1)

starting


In [19]:
print "entering loop"

while d<datetime.datetime(2015,3,1,17):
    get_unzip(d,fo,fo2)  #gets the gz file at wikipedia and unzips it
    print "it is unzipped"
        
    #reading wikiviews into df and keeping the english entries and reasonably larger pages
    #allwiki=pd.read_csv(fo2,delim_whitespace=True)
    #allwiki.columns=['lang','article','views','size']
    #enwiki=allwiki[allwiki['lang']=='en']
    #enwiki=enwiki[enwiki['size']>=50000]
    enwiki=csv_to_reduceddf(fo2) #takes the unzipped file and pairs it down and puts it into a df
    print "english entries separated"
    
    print date
    d+=dt
    

entering loop
pagecounts-20150301-160000.gz
it is unzipped
english entries separated
2015030116


In [16]:
print "here"
#searching each article for the various topics queried and put the results in views
date='%4d%02d%02d%02d'%(d.year,d.month,d.day,d.hour)
dic={}
views=[]
with con:
    cur = con.cursor()
    sql="SELECT `Id`,`topic_label`,`topic_string` FROM `topics`;"
    cur.execute(sql)
    for row in cur:
        tops=row[2].rstrip().split()
        twiki=enwiki[np.logical_and.reduce([enwiki['article'].str.contains(top,case=False) for top in tops ])==True]
        t=row[2].rstrip()
        dic[t]=twiki['views'].sum(axis=0)
    #print row[0],dic[t],date
    views.append([row[0],dic[t],date])
print "pageviews sorted"
        
    #
with con:
    cur = con.cursor()
    for view in views:
        sql="INSERT INTO `page_views` (`topic_id`,`count`,`date`) VALUES (%s,%s,%s);"
        cur.execute(sql,view)
        
print date
d+=dt

entering loop
pagecounts-20150301-160000.gz
it is unzipped


KeyboardInterrupt: 

In [12]:
print "entering loop"

#while d<datetime.datetime(2015,3,1,17):
if True:
    #getting an unzipping gz'ed wikiviews file
    #wikiurl="http://dumps.wikimedia.org/other/pagecounts-raw/%4d/%4d-%02d/"%(d.year,d.year,d.month)
    #gzfile="pagecounts-%4d%02d%02d-%02d0000.gz"%(d.year,d.month,d.day,d.hour)
    #print gzfile
    #fn=wikiurl+gzfile
    #urllib.urlretrieve(fn,fo)
    #!gunzip -c $fo>$fo2
    get_unzip(d,fo,fo2)
    print "it is unzipped"
    
    #reading wikiviews into df and keeping the english entries and reasonably larger pages
    allwiki=pd.read_csv(fo2,delim_whitespace=True)
    allwiki.columns=['lang','article','views','size']
    enwiki=allwiki[allwiki['lang']=='en']
    enwiki=enwiki[enwiki['size']>=50000]
    print "english entries separated"
    
    #searching each article for the various topics queried and put the results in views
    date='%4d%02d%02d%02d'%(d.year,d.month,d.day,d.hour)
    dic={}
    views=[]
    with con:
        cur = con.cursor()
        sql="SELECT `Id`,`topic_label`,`topic_string` FROM `topics`;"
        cur.execute(sql)
        for row in cur:
            tops=row[2].rstrip().split()
            twiki=enwiki[np.logical_and.reduce([enwiki['article'].str.contains(top,case=False) for top in tops ])==True]
            t=row[2].rstrip()
            dic[t]=twiki['views'].sum(axis=0)
        #print row[0],dic[t],date
        views.append([row[0],dic[t],date])
    print "pageviews sorted"
        
    #
    with con:
        cur = con.cursor()
        for view in views:
            sql="INSERT INTO `page_views` (`topic_id`,`count`,`date`) VALUES (%s,%s,%s);"
            cur.execute(sql,view)
        
    print date
    d+=dt

entering loop
pagecounts-20150301-160000.gz
it is unzipped


KeyboardInterrupt: 